In [0]:


!pip3 install box2d-py

In [0]:
import gym
import numpy as np
import time
import tensorflow as tf
import multiprocessing
import threading
from tqdm import tqdm_notebook as tqdm

env=gym.make('BipedalWalkerHardcore-v2').unwrapped

In [0]:
# number of worker agents
no_of_workers = multiprocessing.cpu_count() 

# maximum number of steps per episode
no_of_ep_steps = 20000 

# total number of episodes
no_of_episodes = 50000 


# sets how often the global network should be updated
update_global = 10

# discount factor
gamma = 0.90 

# entropy factor
entropy_beta = 0.01 

# learning rate for actor
lr_a = 0.0001 

# learning rate for critic
lr_c = 0.001 

# boolean for rendering the environment
render=False 

# directory for storing logs
log_dir = 'logs'

# number of frames
no_of_frames=4

In [0]:
action = env.action_space.sample()
obs, reward, done, info = env.step(action)

In [0]:
obs

array([ 0.00245752, -0.00692399,  0.00639904,  0.01958581, -0.29071033,
       -0.71053505,  1.47233772,  0.99359353,  1.        ,  0.30071497,
       -0.01759427,  0.16330874,  0.33355518,  1.        ,  0.45289576,
        0.45803908,  0.47406936,  0.50296766,  0.5487414 ,  0.61897314,
        0.72858512,  0.91021329,  1.        ,  1.        ])

In [0]:
# no of states
no_of_states=env.observation_space.shape[0]
# no of actions
no_of_actions=env.action_space.shape[0]
# range of action
action_bound=[env.action_space.low,env.action_space.high]

In [0]:
class ActorCritic:
  
  def __init__(self,sess,scope,globalAC=None):
    # first we intialize the session and the optimizer for both
    # actor and critic networks
    self.sess=sess
    
    self.actor_optimizer=tf.train.RMSPropOptimizer(lr_a,name='RMSPropA')
    self.critic_optimizer=tf.train.RMSPropOptimizer(lr_c,name='RMSPropC')
    
    # if our network is global then,
    if scope=='global':
      with tf.variable_scope(scope):
        # initialize states
        self.state=tf.placeholder(tf.float32,[None,no_of_frames,no_of_states],'S')
        
        # get the parameters of actor and critic network
        self.actor_params,self.critic_params=self.build_net(scope)[-2:]
        
        print('Global Network created!!')
        
    # if our network is local
    else:
      with tf.variable_scope(scope):
        
        print('Worker created!!')
        # initialize state
        self.state=tf.placeholder(tf.float32,[None,no_of_frames,no_of_states],'S')
      
        # initialize action
        self.action=tf.placeholder(tf.float32,[None,no_of_actions],'A')
        # initialize target value
        self.v_target=tf.placeholder(tf.float32,[None,1],'VTarget')
        # Since we are in continous action space, we will calculate 
        # mean and variance for chosing action
        
        mean,var,self.v,self.actor_params,self.critic_params=self.build_net(scope) 
        
        # calculate td error that is difference between v_target and v
        td=tf.subtract(self.v_target,self.v,name='TD_error')
        
        # minimize the TD error
        with tf.name_scope('critic_loss'):
          
          self.critic_loss=tf.reduce_mean(tf.square(td))
        
        # update the mean and variance value
        with tf.name_scope('wrap_action'):
          
          mean,var=mean*action_bound[1],var+1e-4
          
        # generating gaussian distribution
        normal_dist=tf.contrib.distributions.Normal(mean,var)
        
        
        # calculate actor loss
        with tf.name_scope('actor_loss'):
          
          # calculate first term of loss which is log(pi(s))
          log_prob=normal_dist.log_prob(self.action)
          exp_v=log_prob*td
          
          # calculate entropy from our action distribution for exploration
          entropy=normal_dist.entropy()
          
          # final loss
          self.exp_v=exp_v+entropy_beta*entropy
          
          # minimize the loss
          self.actor_loss=tf.reduce_mean(-self.exp_v)
          
        # now we choose action by drawing from distribution
        # and clipping it between action bounds
        
        with tf.name_scope('choose_action'):
          self.A=tf.clip_by_value(tf.squeeze(normal_dist.sample(1),axis=0),
                                  action_bound[0],action_bound[1])
          
        # calculate gradients for both actor and critic networks
        
        with tf.name_scope('local_gradients'):
          
          self.actor_gradients=tf.gradients(self.actor_loss,self.actor_params)
          self.critic_gradients=tf.gradients(self.critic_loss,self.critic_params)
          
     
    # now lets update our global_network weights
      with tf.name_scope('sync'):
      
        # pull the global network weights to the local networks
        with tf.name_scope('pull'):
          self.pull_actor_params=[lp.assign(gp) for lp,gp in zip(self.actor_params
                                                              ,globalAC.actor_params)]
          self.pull_critic_params=[lp.assign(gp) for lp,gp in zip(self.critic_params,
                                                               globalAC.critic_params)]
        
     # push the local gradients to global network
        with tf.name_scope('push'):
          self.update_actor=self.actor_optimizer.apply_gradients(zip(self.actor_gradients,globalAC.actor_params))
        
          self.update_critic=self.critic_optimizer.apply_gradients(zip(self.critic_gradients,globalAC.critic_params))
          
        
        
          
  def build_net(self,scope):
    # function to build actor and critic networks
    # intialize weights
    weighs_initalizer=tf.random_normal_initializer(0.,.1)
    
    with tf.variable_scope('actor'):
      
      l_a=tf.layers.conv1d(self.state,filters=8,kernel_size=5,padding='same',activation='relu',
                          kernel_initializer=weighs_initalizer)
      l_a=tf.layers.conv1d(l_a,filters=16,kernel_size=3,padding='same',activation='relu',
                          kernel_initializer=weighs_initalizer)
      l_a=tf.layers.flatten(l_a)
      l_a=tf.layers.dense(l_a,128,activation='relu',kernel_initializer=weighs_initalizer)
      
      mean=tf.layers.dense(l_a,no_of_actions,tf.nn.tanh,name='mean',kernel_initializer=weighs_initalizer)
      
      var=tf.layers.dense(l_a,no_of_actions,tf.nn.softplus,name='variance',kernel_initializer=weighs_initalizer)
      
    with tf.variable_scope('critic'):
      
      l_c=tf.layers.conv1d(self.state,filters=8,kernel_size=5,padding='same',activation='relu',
                          kernel_initializer=weighs_initalizer)
      l_c=tf.layers.conv1d(l_c,filters=16,kernel_size=3,padding='same',activation='relu',
                          kernel_initializer=weighs_initalizer)
      l_c=tf.layers.flatten(l_c)
      l_c=tf.layers.dense(l_c,128,activation='relu',kernel_initializer=weighs_initalizer)
      
      l_c=tf.layers.dense(l_c,64,kernel_initializer=weighs_initalizer)
      l_c=tf.layers.dense(l_c,32,kernel_initializer=weighs_initalizer)
      
      v=tf.layers.dense(l_c,1,name='v',kernel_initializer=weighs_initalizer)
      
      a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
      c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        
      return mean, var, v, a_params, c_params
      
      
  def choose_action(self,state):   
    # select action
    
    state=state[np.newaxis,:]
    return self.sess.run(self.A,{self.state:state})[0]
  
  def pull_global(self):
    # copy the global network parameters to local network
    self.sess.run([self.pull_actor_params,self.pull_critic_params])
    
  def update_global(self,feed_dict):
    # update global network parameters
    self.sess.run([self.update_actor,self.update_critic],feed_dict)

In [0]:
env.action_space.high

array([1., 1., 1., 1.], dtype=float32)

In [0]:
class Worker:
  
  def __init__(self,sess,name,globalAC):
    # initialize environment for each worker
    self.env=gym.make('BipedalWalkerHardcore-v2').unwrapped
    self.name=name
    
    # create ActorCritic agent for each worker
    self.AC=ActorCritic(sess,name,globalAC)
    self.sess=sess
    
  def work(self):
    # declare global variables
    global global_rewards,global_episodes
    total_step=1
    
    # replay_memory
    buffer_s,buffer_a,buffer_r=[],[],[]
    #print(type(buffer_s))
    # coordinator implements a simple mechanism to coordinate the termination of a set of threads
    # loop if the coordinator is active and global episode 
    # is less tan maximum no of episodes
    
    while not coord.should_stop() and global_episodes< no_of_episodes:
      
      
      
      # initialize the environment by resetting
      
      s=self.env.reset()
      s=np.array([np.zeros(no_of_states),np.zeros(no_of_states),np.zeros(no_of_states),s])
      # store the episodic reward
      episode_reward=0
      
      for ep_t in tqdm(range(no_of_ep_steps)):
        
        if self.name == 'W_0' and render:
          self.env.render()
        # choose the action based on the policy
        a=self.AC.choose_action(s)
        
        # execute action, recieve reward , move to next state
        
        next_state_temp,r,done,info=self.env.step(a)
        
        next_state=np.array([np.zeros(no_of_states),np.zeros(no_of_states),np.zeros(no_of_states),np.zeros(no_of_states)])
        next_state[0]=s[1]
        next_state[1]=s[2]
        next_state[2]=s[3]
        next_state[3]=next_state_temp
        
        if ep_t==no_of_ep_steps-1:
          done=True
        
        
        episode_reward+=r
        
        # store the state,action and rewards in buffer
        buffer_s.append(s)
        buffer_a.append(a)
        buffer_r.append(r)
        
        # we update the global network after particular time step
        if total_step% update_global==0 or done:
          
          if done:
            v_s_=0
          else:
            v_s_=self.sess.run(self.AC.v,{self.AC.state:next_state[np.newaxis,:]})[0,0]
            
          # buffer for target v
        
          buffer_value_target=[]
        
          for r in buffer_r[::-1]:
            v_s=r+gamma*v_s_
            buffer_value_target.append(v_s_)
          
          buffer_value_target.reverse()
          buffer_s=np.array(buffer_s)
          
          buffer_a,buffer_value_target=np.vstack(buffer_a),np.vstack(buffer_value_target)
           
          feed_dict={
              self.AC.state:buffer_s,
              self.AC.action:buffer_a,
              self.AC.v_target:buffer_value_target
          }
          
          self.AC.update_global(feed_dict)
          buffer_s,buffer_a,buffer_r=[],[],[]
                                                                        
        s=next_state
        total_step+=1
        
        if done:
          if len(global_rewards)<5:
            global_rewards.append(episode_reward)
            
          else:
            global_rewards.append(episode_reward)
            global_rewards[-1]=((np.mean(global_rewards[-5:])))
            
          global_episodes+=1
          
          break
        
    

In [0]:
tf.reset_default_graph()
# create a list for global rewards and episodes
global_rewards=[]
global_episodes=0

# tensorflow session
sess=tf.Session()
#tf.reset_default_graph()

with tf.device('/cpu:0'):
  
  # create an instance to our ActorCritic Class
  
  global_ac=ActorCritic(sess,'global')
  workers=[]
  
  # loop for each workers
  for i in range(no_of_workers):
    worker_name='W_%i' % i
    workers.append(Worker(sess,worker_name,global_ac))
    
coord=tf.train.Coordinator()    
sess.run(tf.global_variables_initializer())

# write summary to tensorboard

tf.summary.FileWriter(log_dir,sess.graph)

worker_threads=[]

for worker in workers:
  
  job=lambda: worker.work()
  t=threading.Thread(target=job)
  t.start()
  worker_threads.append(t)
  
coord.join(worker_threads)  

W0809 10:12:03.495643 140294292658048 deprecation.py:323] From <ipython-input-7-0666bf3c8484>:114: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
W0809 10:12:03.645514 140294292658048 deprecation.py:323] From <ipython-input-7-0666bf3c8484>:117: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0809 10:12:03.902801 140294292658048 deprecation.py:323] From <ipython-input-7-0666bf3c8484>:118: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Global Network created!!
Worker created!!


W0809 10:12:05.751983 140294292658048 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0809 10:12:05.753538 140294292658048 deprecation.py:323] From <ipython-input-7-0666bf3c8484>:53: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W0809 10:12:05.757454 140294292658048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/di

Worker created!!


Buffered data was truncated after reaching the output size limit.

In [0]:
a=global_ac.actor_params
b=global_ac.critic_params